In [1]:
from camelot import read_pdf,plot
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np

In [2]:
pdf_file = open('volume16.pdf','rb')
NumPages = PdfFileReader(pdf_file).getNumPages()

Xref table not zero-indexed. ID numbers for objects will be corrected.


In [8]:
content={}
content[0]=[]
content[1]=[]
for page in range(80,81): # range(1,NumPages+1)
    tables = read_pdf('volume16.pdf',pages=f'{page}',flavor = 'stream', # pages 80/92
            table_areas=['40,800,545,50'],columns=['63,155,213,274,323,381,431,490'],split_text=True
        )
    if len(tables) == 1:
        # extract state
        state=tables[0].df.iloc[0].tolist() 
        while '' in state: state.remove('')
        state=''.join(state)
        if state not in ['KUALA LUMPUR','SELANGOR','JOHOR','PULAU PINANG','NEGERI SEMBILAN','PERAK','MELAKA','KEDAH','PAHANG','TERENGGANU','KELANTAN','PERLIS','SABAH','SARAWAK']:
            continue
        column_name={}
        # extract column_name_0
        column_name[0]=tables[0].df.iloc[1].tolist()
        column_name[0][0]='Record ID'
        # extract column_name_1
        column_name[1]=tables[0].df.iloc[2].tolist()
        column_name[1][1]='Location 2'
        column_name[1].append('State')
        column_name[1].append('Property Type')
        # extract property_type
        tmp_prop_type=tables[0].df.iloc[3].tolist() 
        while '' in tmp_prop_type: tmp_prop_type.remove('')
        if len(tmp_prop_type) == 1:
            property_type = tmp_prop_type[0]
            tmp_content = tables[0].df.iloc[4:]
        else:
            tmp_content = tables[0].df.iloc[3:]
        rowNum=len(tmp_content.index)
        column_name_index=0
        for x in range(rowNum):
            tmp_prop_type=tmp_content.iloc[x].tolist()
            while '' in tmp_prop_type: tmp_prop_type.remove('')
            if len(tmp_prop_type) == 1 and tmp_prop_type[0].isupper() == True:
                property_type = tmp_prop_type[0]
                column_name_index=0
            elif len(tmp_prop_type) == 1 and tmp_prop_type[0].isupper() == False:
                pass
            else:
                tmp_content_app=tmp_content.iloc[x].tolist()
                if tmp_content_app[0]=='' and column_name_index%2 == 1:
                    tmp_content_app.append(state)
                    tmp_content_app.append(property_type)
                if tmp_content_app[0]=='' and column_name_index%2 == 0:
                    continue
                content[column_name_index%2].append(tmp_content_app)
                column_name_index+=1
    else:
        print('Duplicate detected!!!')
content_0_df=pd.DataFrame(content[0],columns=column_name[0])
content_1_df=pd.DataFrame(content[1],columns=column_name[1])
final_df=pd.concat([content_0_df,content_1_df],axis=1)
print(final_df)

   Record ID                Location        Mukim   Lot No  Land Area  \
0          1          Jln Perusahaan            1  PT 1430   4,059 mp   
1          2      Solok Perusahaan 3            1    PT 18   1,766 mp   
2          3       Perai Ind. Estate            1  PT 2827  19,749 mp   
3          4           Lrg Jelawat 8            1     5342   1,685 mp   
4          5          Jln Perusahaan            1    PT 25   1,995 mp   
5          6          Jln Perusahaan            1     1347   7,870 mp   
6          7           Lrg Jelawat 2            1  PT 4078   8,420 mp   
7          8           Lrg Jelawat 8            1     5343   2,057 mp   
8          9       Lrg Perusahaan 10            1      245  10,117 mp   
9         10        Lrg Perusahaan 2            1  PT 3968  10,631 mp   
10        11        Lrg Perusahaan 1            1      684   4,416 mp   
11        12       Lebuhraya Kg Jawa           12  PT 5614  34,204 mp   
12        13   Lintang Bayan Lepas 7           12  